In [ ]:
import scipy.signal as sig
import control as ctrl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

import fixpt
from biquad_filters import BiQuad, BiQuadDF1, BiQuadCoupled

Конфигурация блока

In [ ]:
# период дискретизации (с)
T = 1/20000
# конфигурация блока DF1
df1_config = BiQuadDF1.Config(BASE_FREQ = 20000,
                              SIGNAL_NBITS = 18,
                              A_NBITS = 18, 
                              A1_FRACBITS = 16,
                              A2_FRACBITS = 17, 
                              B_NBITS = 27,
                              B_FRACBITS = 24,
                              STATE_NBITS = 27,
                              STATE_FRACBITS = 9,
                              QUANT_POLICY = fixpt.QuantPolicy.TruncateToZero)
# конфигурация блока Coupled
coup_config = BiQuadCoupled.Config(BASE_FREQ = 20000,
                                   SIGNAL_NBITS = 18,
                                   ALPHA_NBITS = 18, 
                                   ALPHA_FRACBITS = 17, 
                                   Q_NBITS = 18,
                                   Q_FRACBITS = 16,
                                   STATE_NBITS = 27,
                                   STATE_FRACBITS = 9,
                                   QUANT_POLICY = fixpt.QuantPolicy.TruncateToZero)

Процедуры синтеза фильтров

In [ ]:
def lowpass1(f_cut, T):
    ''' Lowpass 1nd order butterwort filter synthesis.
    '''
    a = np.exp(-2*np.pi*f_cut*T)
    return [1-a, 0, 0], [1, -a, 0]
    
def lowpass2(f_cut, T):
    ''' Lowpass 2nd order butterwort filter synthesis.
    '''
    f_nyquist = 0.5/T
    b, a = sig.butter(2, f_cut/f_nyquist)
    return b, a

def notch2(f_nominal, f_width, T):
    ''' Two parameters Notch (band-stop) filter synthesis.

        Calculate coffecients of notch filter with stop-band center at f_nominal (Hz) with width f_width (Hz).
        T is discretization period (s). Returns numerator and denominatror (b, a) of transfer function. 
    '''
    wn = 2.0*np.pi * f_nominal * T
    wc = 2.0*np.pi * f_width * T
    a = 1.0 / np.cos(wc / 2.0) - np.tan(wc / 2.0)
    K = (1.0 - 2.0*a*np.cos(wn) + a**2) / (2.0 - 2.0*np.cos(wn))
    b = [K, -2.0*K*np.cos(wn), K]
    a = [1.0, -2.0*a*np.cos(wn), a**2]
    return b, a

def notch3(f_nominal, f_width, L_stop, T):
    ''' Notch (band-stop) filter synthesis.

        Calculate coffecients of notch filter with stop-band center at f_nominal (Hz) with width f_width (Hz).
        L_stop (dB) is filter attenuation at f_nominal frequency. It must be lower then zero.
        T is discretization period (s). Returns numerator and denominatror (b, a) of transfer function. 
    '''
    # check parameters
    if T <= 0.0:
        raise ValueError('T must be posititve')
    f_nyquist = 1/(2*T)
    if f_nominal < 0.0 or f_nominal > f_nyquist:
        raise ValueError('f_nominal must be in interval [0.0, 1/(2*T)]')
    if f_width < 0.0 or f_width >= 2*f_nominal:
        raise ValueError('f_width must be in interval [0.0, 2*f_nominal]')
    if L_stop >= 0.0:
        raise ValueError('L_stop must be negative.')
    # convert Hz to rad/s
    Wn = 2*np.pi*f_nominal
    Wc = 2*np.pi*f_width
    # notch parameters
    b = 10**(L_stop/20)
    p = (Wn - Wc/2)/Wn
    a = 0.5 * np.sqrt( (p - 1/p)**2 / (1 - 2*b**2) )
    # tf numerator and denominator
    den = np.array([1, - 2*np.exp(-a*Wn*T)*np.cos(Wn*T*np.sqrt(1-a**2)), np.exp(-2*a*Wn*T)])
    num = np.array([1, - 2*np.exp(-a*b*Wn*T)*np.cos(Wn*T*np.sqrt(1-a**2*b**2)), np.exp(-2*a*b*Wn*T)])
    num *= np.sum(den) / np.sum(num)
    return num, den

Тестовый сигнал

In [ ]:
# Тестовый сигнал
t = np.arange(0.0, 2.0, T)
u = 100 * sig.chirp(t, 1.0, t[-1], 200.0, method='linear')
u[-4000:] = 0

Сравнение поведений реализация фильтров на тестовом сигнале: 
* реализации с плавающими точками (эталон)
* реализации DF1
* реализации Coupled

In [ ]:
# выбор фильтра
filter_type = 'notch' # 'lowpass1', 'lowpass2', 'highpass1', 'highpass2', 'notch'
f_nominal = 40
f_width = 5
L_stop = -10
f_start = 50
f_end = 5000

# расчет коэффициентов фильтров
f_nyquist = 1 / (2*T)
if filter_type == 'lowpass1':
    b, a = lowpass1(f_nominal, T)
elif filter_type == 'lowpass2':
    b, a = lowpass2(f_nominal, T)
elif filter_type == 'highpass1':
    b, a = lowpass1(f_nominal, T)
    b = np.array(a) - np.array(b)
elif filter_type == 'highpass2':
    b, a = lowpass2(f_nominal, T)
    b = np.array(a) - np.array(b)
elif filter_type in ('bandpass', 'bandstop'):
    b, a = sig.butter(1, [f_start/f_nyquist, f_end/f_nyquist], filter)
elif filter_type == 'notch':
    b, a = notch3(f_nominal, f_width, L_stop, T)
elif filter_type == 'notch_inf':
    b, a = notch2(f_nominal, f_width, T)
else:
    raise ValueError('unknown filter type')

# передаточные функции
Hz = BiQuad(b, a, T)
# DF1
regs_df1 = BiQuadDF1.Registers(df1_config).from_tf(b, a)
Hz_df1 = BiQuadDF1(regs_df1, df1_config)
# Сoupled
regs_coup = BiQuadCoupled.Registers(coup_config)
try:
    regs_coup.from_tf(b, a)
except ValueError:
    pass
Hz_coup = BiQuadCoupled(regs_coup, coup_config)

# моделирование
_, y = Hz.output(u)
_, y_df1 = Hz_df1.output(u)
_, y_coup = Hz_coup.output(u)

# построение графиков
fig, axs = plt.subplots(2, 1, figsize=(12,12))
axs[0].plot(t, y, label='float')
axs[0].plot(t, y_df1, label='DF1')
axs[0].plot(t, y_coup, label='Coupled')
axs[0].set_xlabel('t, s')
axs[0].set_ylabel('y')
axs[0].legend()
axs[0].set_title('Сравнение выходов реализаций фильтров')
axs[1].plot(t, y - y_df1, label='DF1 error')
axs[1].plot(t, y - y_coup, label='Coupled error')
axs[1].set_xlabel('t, s')
axs[1].set_ylabel('e')
axs[1].legend()
axs[1].set_title('Расхождение с реализацие на плавающих точках')

plt.show()

Диаграммы Боде (учитываеют только квантование коэффцицентов

In [ ]:
# диаграммы Боде
fig = plt.figure(figsize=(12,12))
out = ctrl.bode_plot([Hz, Hz_df1, Hz_coup], Hz = True, dB = True, label=('float', 'DF1', 'Coupled'), omega=(2*np.pi, np.pi/T-1), title='Изменение диаграммы Боде из-за квантования коэффициентов')
# omega=2*np.pi*np.logspace(0,np.log10(1/(2*T)),1000, endpoint=False)
#out.axes[0,0].set_ylim((-80, 5))

Шаблон для тестированяи блока
* `u_fpga` вход в raw представлении (int32)
* `y_fpga` выход в raw представлении (int32)

In [ ]:
# инициализируем из HWH-конфигурации 
config = BiQuadDF1.Config(BASE_FREQ = 20000,
                             SIGNAL_NBITS = 18,
                             A_NBITS = hwh..., 
                             B_NBITS = hwh...,
                             STATE_NBITS = hwh...,
                             A1_FRACBITS = hwh..., 
                             A2_FRACBITS = hwh..., 
                             B_FRACBITS = hwh...,
                             STATE_FRACBITS = hwh...,
                             QUANT_POLICY = fixpt.QuantPolicy.TruncateToZero)

# загружаем занчения регистров (raw значения регистров как int32)
regs = BiQuadDF1.Registers(config).from_raw(
                            b0 = axis...register...,
                            b1 = axis...register...,
                            b2 = axis...register...,
                            a1 = axis...register...,
                            a2 = axis...register...)

# объект для моделироваяни
Hz = BiQuadDF1(regs, config)

# моделирвоание
# Следует учесть, что начальное состояние блока ПЛИС и кго модели может ыть разным, поэтому надо 
# * обнулить состояния обоих (ПЛИС и эталона) 
# ИЛИ
# * назначить сотсояние модели из значений регистров ПЛИС используя set_state_raw().
Hz.set_state() # нулевое состояние
_, y_model = Hz.output(u_fpga)

# сравнение
if all(y_model - y_fpga < threshold):
    print('OK')
else:
    print('FAILED')